In [1]:
import pandas as pd
import tqdm.auto as tqdm
import time

## select translator and language
if True:
    from deep_translator import GoogleTranslator
    translate = GoogleTranslator(source='en', target='ko').translate
else:
    import dl_translate as dlt
    from functools import partial
    translate = partial(dlt.TranslationModel().translate, source=dlt.lang.ENGLISH, target=dlt.lang.KOREAN)

In [2]:
translate("Sigil Of The Heart")

'마음의 인장'

In [3]:
df = pd.read_csv('newLanguage.tsv', delimiter='\t', index_col=False, names=['id', 'value'])
df_new = df.copy()
df_new['translated']=None
df_new.head()

,id,value,translated
0,Death Trash Language File,NaN,None
1,File Format Version:,1.0,None
2,Language Id:,NewLanguage,None
3,------------------------------------------,NaN,None
4,#NewGame:,New Game,None


In [19]:
name_token="**forename**"
name_token_translated=translate(name_token)

queue = []
for i in range(len(df_new)):
    
    if not isinstance(df_new.loc[i].value, str):
        continue
    if '<' in df_new.loc[i].value:
        continue
    if len(df_new.loc[i].value)<2:
        continue
    if not df_new.loc[i].translated==None:
        continue
    
    queue.append(i)
    
for i in tqdm.tqdm(queue):

    sequence = df_new.loc[i].value

    isDialogueItem = sequence[0] == '[' and sequence[-1] == ']' 
    
    if isDialogueItem:
        sequence = sequence[1:-1]
        
    try:
        translated = translate(sequence)
    except:
        continue
    
    try:
        translated = translated.replace(name_token_translated, name_token)
        ## Add original sequence
        translated = '{}({})'.format(translated, sequence)
    except:
        translated = sequence
        
    if isDialogueItem:
        translated = "[{}]".format(translated)
        
    df_new.loc[i].translated = translated
    time.sleep(0.1)
        

  0%|          | 0/1900 [00:00<?, ?it/s]

In [20]:
df_new.to_csv('newLanguage_translated.tsv', index=None, sep='\t', header=None)